In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
# Cargar los datos.
data = pd.read_csv('db/EarlyRetirementPrediction.csv')
data.head()

,Retire Before 65 Years Old,Gender,Desease,Education Level,Marital Status,Monthly Income,Employee/Employer,Dependants,Unemployment Rate,Stock Market,Credit Score,Government Bonds Return
0,1,1,1,3,0,18000,0,4,3.444876,24066.16667,699,7.715833
1,0,1,0,3,0,32000,0,1,5.023797,42063.16667,639,5.414833
2,0,1,1,4,0,20000,0,2,5.355220,30100.50000,610,7.962750
3,0,0,1,4,1,35000,1,1,3.843570,24066.16667,649,7.715833
4,0,0,1,3,0,27000,0,1,4.949226,19439.00000,638,8.024167


In [3]:
# Definir características y etiquetas.

# Características.
X = data.drop(columns=['Retire Before 65 Years Old']).values

# Etiqueta a predecir (objetivo).
y = data['Retire Before 65 Years Old'].values.reshape(-1, 1)

In [4]:
# Escalar los datos (normalización: media = 0, desviación típica = 1).
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [5]:
# Asignar los pesos de forma aleatoria para la capa oculta (4 neuronas)
# y la capa de salida.
np.random.seed(42)
n_entrada = X_scaled.shape[1]   # Número de características de entrada.
n_oculta = 4    # Número de neuronas en la capa oculta.
n_salida = 1    # Neurona de salida.

# Pesos y sesgos de la capa oculta.
pesos_entrada_oculta = np.random.randn(n_entrada, n_oculta)
sesgos_oculta = np.random.randn(1, n_oculta)

# Pesos y sesgos de la capa de salida.
pesos_oculta_salida = np.random.randn(n_oculta, n_salida)
sesgos_salida = np.random.randn(1, n_salida)

In [6]:
# Parámetros del entrenamiento.
learning_rate = 0.01    # Tasa de aprendizaje.
epochs = 10 # Número de épocas.
n_samples = X_scaled.shape[0]   # Número de observaciones.

In [7]:
# Función de activación ReLU.
def relu(x):
    return np.maximum(0, x)

# Derivada de ReLU
def relu_derivada(x):
    return np.where(x > 0, 1, 0)

# Función sigmoide para la capa de salida.
def sigmoide(x):
    return 1 / (1 + np.exp(-x))

# Derivada de la función sigmoide.
def sigmoide_derivada(y):
    return y * (1 - y)

In [8]:
# Función para la propagación hacia adelante.
def propagacion_hacia_adelante(X_i, pesos_entrada_oculta, sesgos_oculta,
                               pesos_oculta_salida, sesgos_salida):
    # Capa oculta
    z_oculta = np.dot(X_i, pesos_entrada_oculta) + sesgos_oculta
    a_oculta = relu(z_oculta)   # Activación ReLU en la capa oculta.

    # Capa de salida
    z_salida = np.dot(a_oculta, pesos_oculta_salida) + sesgos_salida
    prediccion = sigmoide(z_salida) # Activación sigmoide en la capa de salida.

    return z_oculta, a_oculta, z_salida, prediccion

In [9]:
# Función para la propagación hacia atrás.
def propagacion_hacia_atras(y_i, z_oculta, prediccion, pesos_oculta_salida):
    # Error en la capa de salida.
    error_salida = prediccion - y_i
    delta_salida = error_salida * sigmoide_derivada(prediccion)

    # Error en la capa oculta.
    error_oculta = np.dot(delta_salida, pesos_oculta_salida.T)
    delta_oculta = error_oculta * relu_derivada(z_oculta)

    return delta_salida, delta_oculta

In [10]:
# Entrenamiento de la red neuronal.
for epoch in range(epochs):
    for i in range(n_samples):
        # Obtener una sola observación (tamaño del lote = 1)
        X_i = X_scaled[i].reshape(1, -1)    # Convertir a vector fila
        y_i = y[i].reshape(1, -1)

        # --- Propagación hacia adelante ---
        z_oculta, a_oculta, z_salida, prediccion = propagacion_hacia_adelante(
            X_i, pesos_entrada_oculta, sesgos_oculta,
            pesos_oculta_salida, sesgos_salida)
        
        # --- Propagación hacia atrás ---
        delta_salida, delta_oculta = propagacion_hacia_atras(
            y_i, z_oculta, prediccion, pesos_oculta_salida)
        
        # --- Actualización de los pesos ---

        # Actualizar pesos y sesgos de la capa de salida.
        pesos_oculta_salida -= learning_rate * np.dot(a_oculta.T, delta_salida)
        sesgos_salida -= learning_rate * delta_salida

        # Actualizar pesos y sesgos de la capa oculta.
        pesos_entrada_oculta -= learning_rate * np.dot(X_i.T, delta_oculta)
        sesgos_oculta -= learning_rate * delta_oculta

    # Imprimir los pesos al final de cada época.
    print(f"Época {epoch+1}, pesos finales de la capa de salida:\n",
          pesos_oculta_salida.flatten())

# Pesos finales después de 10 épocas.
print("Pesos finales después de 10 épocas (capa oculta a salida):\n",
      pesos_oculta_salida.flatten())
print("Pesos finales después de 10 épocas (entrada a oculta):\n",
      pesos_entrada_oculta.flatten())

Época 1, pesos finales de la capa de salida:
 [ 0.3403547  -1.685444    0.10618938  0.10920205]
Época 2, pesos finales de la capa de salida:
 [ 0.3175179  -1.57468847 -0.05550904  0.14722628]
Época 3, pesos finales de la capa de salida:
 [ 0.30109768 -1.4523696  -0.11482187  0.16517811]
Época 4, pesos finales de la capa de salida:
 [ 0.29074566 -1.33329647 -0.13910719  0.17337139]
Época 5, pesos finales de la capa de salida:
 [ 0.28634388 -1.22779559 -0.15681646  0.17130139]
Época 6, pesos finales de la capa de salida:
 [ 0.286027   -1.14370807 -0.17645115  0.17004666]
Época 7, pesos finales de la capa de salida:
 [ 0.28772504 -1.07821094 -0.19826639  0.17576983]
Época 8, pesos finales de la capa de salida:
 [ 0.29163682 -1.02867687 -0.21916126  0.18671408]
Época 9, pesos finales de la capa de salida:
 [ 0.29932492 -0.99074916 -0.23801711  0.20160476]
Época 10, pesos finales de la capa de salida:
 [ 0.30959688 -0.95983722 -0.25588503  0.2179371 ]
Pesos finales después de 10 épocas (cap

In [11]:
# Predicciones para todo el conjunto de entrenamiento.
predicciones = []
for i in range(n_samples):
    X_i = X_scaled[i].reshape(1, -1)
    _, _, _, prediccion = propagacion_hacia_adelante(
        X_i, pesos_entrada_oculta, sesgos_oculta,
        pesos_oculta_salida, sesgos_salida)
    predicciones.append(prediccion.flatten()[0])

# Convertir las predicciones a una clase (0 o 1) usando un umbral de 0.5.
predicciones_clase = (np.array(predicciones) >= 0.5).astype(int)

# Calcular la exactitud del modelo
exactitud = accuracy_score(y, predicciones_clase)
print(f"Exactitud del modelo: {exactitud:.2f}")

Exactitud del modelo: 0.68
